<a href="https://colab.research.google.com/github/harshitavishnoi/Ai_career_coach/blob/main/Ai_Career_Coach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.4/270.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.6 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from gradientai import Gradient
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
import csv

#set gradient environment variables
os.environ['GRADIENT_ACCESS_TOKEN'] = 'kcJ5aaga3NpVrR8G2aGtHQY3wpuCa9p6'
os.environ['GRADIENT_WORKSPACE_ID'] = 'c6b0df04-84f1-4542-8616-b8dc41d614d5_workspace'

#define the path to dataset
data_path = '/content/drive/MyDrive/my_projects/datasets/truncated_career_recommender_dataset.csv'

#initialize the gradient
gradient = Gradient()

#load the dataset
print("Loading and Formatting the data.....")
formatted_data = []
with open(data_path,encoding = "utf-8-sig") as f:
  dataset_data = csv.DictReader(f,delimiter = ",")
  for row in dataset_data:
    #construct the prompt from the user's data
    user_data = f"Interests: {row['Interests']},Skills: {row['Skills']}, Degree: {row['Undergraduate Course']}, Working: {row['Employment Status']}"
    #the response is the career path
    career_response = row['Career Path']

    #format the data for fine-tuning
    formatted_entry = {
        'inputs': f'### User Data:\n{user_data}\n\n### Suggested Career Path:',
        'response' : career_response
    }
    formatted_data.append(formatted_entry)

#get the base model from gradient
base = gradient.get_base_model(base_model_slug = "nous-hermes2")

#create a model adapter for fine-tuning
new_model_adapter = base.create_model_adapter(name='ai_career_chatbot')

#fine-tune the model adapter in chunks to prevent memory issues
print('fine-tuning model adapter...')
chunk_lines = 20
total_chunks = [formatted_data[x:x+chunk_lines] for x in range(0,len(formatted_data),chunk_lines)]
for i,chunk in enumerate(total_chunks):
  try:
    print(f"fine-tuning chunk {i+1} of {len(total_chunks)}")
    new_model_adapter.fine_tune(samples = chunk)
  except Exception as error:
    print(f"error in fine-tuning chunk {i+1}: {error}")


Loading and Formatting the data.....
fine-tuning model adapter...
fine-tuning chunk 1 of 5
fine-tuning chunk 2 of 5
fine-tuning chunk 3 of 5
fine-tuning chunk 4 of 5
fine-tuning chunk 5 of 5


In [ ]:
user_query = "Interests: designing, Skills: java, Degre: B.Sc in Computer Science, Working: No"
formatted_query = f"### User Data: \n{user_query}\n\n### Suggested Career Path"
response = new_model_adapter.complete(query=formatted_query, max_generated_token_count=100)
print(f"> {user_query}\n> {response.generated_output}")

> Interests: designing, Skills: java, Degre: B.Sc in Computer Science, Working: No
>  Based on your interests and skills, a suggested career path for you could be a User Experience (UX) Designer or User Interface (UI) Designer. As a UX/UI designer, you would be responsible for designing and enhancing the user experience of digital products, such as websites, mobile applications, and software. You would use your skills in Java to create prototypes, wireframes, and mockups, and work closely with developers and other team members to
